In [288]:
import logging

In [289]:
def print_graph(graph):
    for k, v in graph.iteritems():        
        logging.debug( k, ':', v)

In [290]:
def duplicate_and_transform_graph(graph, T):
    l = len(graph)
    new_graph = {}
    offset = max(graph.keys()) + 1
    to_duplicant = lambda x: None if x is None else x + offset
    for k, (formula, (cyclic, neighbours)) in graph.iteritems():
        new_graph[k] = formula, (cyclic, neighbours)
        new_graph[to_duplicant(k)] = T + formula, (cyclic, [to_duplicant(x) for x in neighbours])  
    return new_graph

In [291]:
from copy import copy
def clone_graph(graph):
    return {k: (f, (m, copy(ns))) for k, (f, (m, ns)) in graph.iteritems()}

In [292]:
# def apply_prefix_rules(graph, rules):
#     new_graph = []
#     for formula, (mod, neighbours) in graph:
#         for (prefix, replacement) in rules:
#             if formula.startswith(rule):
#                 new_formula = replacement + formula[len(prefix)+1:]
#             else:
#                 new_formula = formula
#             new_graph.append((new_formula, (mod, neighbours)))

In [293]:
# def intersection_with_indices(l1, l2):
#     common_members = set(l1).intersection(l2)
#     return [(x, l1.index(x), l2.index(x)) for x in common_members]

In [294]:
def pairwise_induction(pairs):
    for (a, b) in pairs:
        if a is not None and b is not None:
            yield (a, b)
        elif a is None:
            yield (b, b)
        else:
            yield (a, a)
            

In [295]:
def pad(l, n, padding_direction=1, v=None):
    from itertools import repeat
    if len(l) > l:
        return l
    padding = list(repeat(v, n - len(l)))
    if padding_direction > 0:
        return l + padding
    else:
        return padding + l

In [296]:
def has_repeating_values(l):
    without_nones = filter(lambda x: x is not None, l)
    return len(l) > len(set(l))

In [297]:
def is_sorted(l):
    return all([x < y for x, y in zip(l, l[1:])])

In [298]:
def padding_zip(l1, l2, padding_direction=1):
    length = max(len(l1), len(l2))
    return zip(pad(l1, length, padding_direction),
               pad(l2, length, padding_direction))

In [299]:
def mirror(tpls):
    if not tpls:
        return None
    (c, l) = tpls
    return (c, [(b, a) for (a, b) in l])

In [300]:
def common_cycle3(s1, s2):
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    if not c1 or not c2:
        #logging.debug( 'Non cyclic')
        return None
    if len(l1) != len(l2):
        #logging.debug( 'Non matching lengths {} {}'.format(l1, l2))
        return None
    common_members = set(l1).intersection(l2)
    if len(common_members) == 0:
        logging.debug( 'No common members: {}, {}'.format(l1, l2))
        return None
    m0 = common_members.pop()
    skew1 = l1.index(m0)
    skew2 = l2.index(m0)
    candidate = list(pairwise_induction([(l1[(i + skew1) % len(l1)], l2[(i + skew2) % len(l2)])
                                         for i in range(len(l1))]))
    if any([x != y and (x in common_members or y in common_members) for x, y in candidate]):
        logging.debug( [x != y and (x in common_members or y in common_members) for x, y in candidate])
        logging.debug( 'Non matching cycles {}'.format(candidate))
        return None
    return (True, candidate)

In [301]:
def aget(l, i):
    return l[i] if i >= 0 and i < len(l) else None
def common_cycle2(s1, s2):
    import re
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    if len(l2) < len(l1) or (c1 and not c2):        
        return common_cycle2(s2, s1)
    # l1 <= l2, 
    # either c1 is open or c2 is closed
    common_numbers = set(l1).intersection(l2)
    def to_letter(n, wild='_'):
        if n > 26:
            raise Exception('Illegal ordinal: {}'.format(n))
        return chr(ord('A') + n) if n in common_numbers else wild
    def to_word(l):
        return ''.join([to_letter(n) for n in l])
    def to_result(match, reverse_lookup, force_closed=False, default=(None)):
        if not match:
            return default
        is_closed = force_closed or len(match.groups()) > 1
        return (is_closed, reverse_lookup[match.start():match.end()], match.re, match.string, match.group(0))
    if c1 and c2:
        # c1 is closed, c2 is closed
        regex = '({0})(?={1})'.format(to_word(l1), to_letter(l1[0]))
        matched_string = '{0}{0}'.format(to_word(l2), to_letter(l2[0]))
        reverse_lookup = l2 + l2
        return to_result(re.search(regex, matched_string), reverse_lookup, True)
    elif c2:
        # c1 is open, c2 is closed
        regex = '({0})({1}?)'.format(to_word(l1), to_letter(l1[0]))
        matched_string = '{0}{0}'.format(to_word(l2))
        reverse_lookup = l2 + l2
        match = re.search(regex, matched_string)
        return to_result(match, reverse_lookup)
    else:
        # c1 is open, c2 is open
        k1, fc1 = [(j, n) for j, n in enumerate(l1) if n in common_numbers][0]
        k2, fc2 = [(j, n) for j, n in enumerate(l2) if n in common_numbers][0]
        if fc1 != fc2:
            k1 = l1.index(fc2)
            while k2 - 1 >= 0 and \
                  k1 - 1 >= 0 and \
                  l1[k1 - 1] not in common_numbers and \
                  l2[k2 - 1] not in common_numbers:
                k1 = k1 - 1
                k2 = k2 - 1
            matched_string = to_word(l2)
            reverse_lookup = l2
            logging.debug( (k1, k2))
            regex = '{0}(_*){1}'.format(to_word(l1[k1:]), to_word(l1[0:k1]))
            return to_result(re.search(regex, matched_string), reverse_lookup, True)
        else:
            # Non cyclic result
            if k2 < k1:
                matched_string = to_word(l2)
                regex = to_word(l1[k1:])
                return (False, l1[0:k1] + l2[k2:])
            else:
                return (False, l2[0:k2] + l1[k1:])
#         closing_regex = '-*'.join([to_letter(n, wild='(_|-*)') for n in l1])
#         closing_matched_string = '{0}{1}{0}'.format(''.join([to_letter(n) for n in l2]),
#                                                     '-' * len(l1))
        
#         match = re.search(closing_regex, closing_matched_string)
#         if not match:
#             return (None, closing_regex, closing_matched_string)
#         if '-' in match.group(0):
#             if not re.search('-_+$', match.group(0)):
#                 # A cycle was closed by a common char
#                 a_common = list(common_numbers)[0]
#                 offset = l1.index(a_common) - l2.index(a_common)
#                 indices = range(0, match.end() - len(re.search('-*', match.group(0)).group(0)))
#                 firsts = [aget(l1, j - offset) for j in indices]
#                 seconds = [aget(l2, j) for j in indices]
#                 return (True, firsts, seconds, match.group(0), match.start(), match.end())
#             # A cycle was 'closed' by a wildcard, not considered a closing of a cycle
#             # Rather, the first sequence extends the second
#             indices = range(0, match.end() - len(re.search('-*', match.group(0)).group(0)))
#             offset = match.start()
#             firsts = [aget(l1, j - offset) for j in indices]
#             seconds = [aget(l2, j) for j in indices]
#             return (False, firsts, seconds, match.group(0), match.start(), match.end())
#         else:            
#             # First sequence is not conained in second
#             return (False, l2, match.group(0), match.start(), match.end())

In [302]:
common_cycle2((False, [1, 4, 10, 9, 2, 6]), (False, [8, 2, 6, 7, 1, 4]))

In [303]:
common_cycle2((False, [1, 4, 9, 2, 6]), (False, [7, 10, 1, 4, 9]))

(False, [7, 10, 1, 4, 9, 2, 6])

In [304]:
import re
re.search('(C)(D)', 'CD').end()
['3'] * 3

['3', '3', '3']

In [305]:
re.search('(A*)(A+)', 'AAAA').groups()

('AAA', 'A')

In [306]:
def common_cycle(s1, s2):
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    if c1 and c2 and len(l1) != len(l2):
        logging.debug( 'Cannot combine closed cycles of different lengths')
    if has_repeating_values(l1):
        raise Exception('Non unique members in cycle 1')
    if has_repeating_values(l2):
        raise Exception('Non unique members in cycle 2')
    tpls = sorted(intersection_with_indices(l1, l2), key= lambda x: x[1:2])
    if len(tpls) == 0:
        logging.debug( 'Cannot combine non intersecting sections')
        return None
    if is_sorted([x[2] for x in tpls]):
        logging.debug( tpls)
        result = list(padding_zip(l1[0:tpls[0][1]], l2[0:tpls[0][2]], padding_direction=-1))
        for prv, tpl in zip(tpls, tpls[1:]):
            if tpl[1] - prv[1] == tpl[2] - prv[2]:
                result = result + list(zip(l1[prv[1]:tpl[1]], l2[prv[2]:tpl[2]]))
            else:
                logging.debug( 'Misatch: {} - {} != {} - {}'.format(tpl[1], prv[1], tpl[2], prv[2]))
                return None
        result = result + padding_zip(l1[tpls[-1][1]:], l2[tpls[-1][2]:])
        return (c1 or c2, list(pairwise_induction(result)))
    result = []
    overboard = False
    for i, prv, tpl in zip(range(len(tpls)), tpls, tpls[1:]):
        if tpl[2] < prv[2]:
            ''' List 2 performed a whole cycle '''
            if overboard:
                raise Exception('Ordering is not cyclic')
            overboard = True
            gap = (tpl[1] - prv[1]) - (tpl[2] + len(l2) - prv[2])
            if c2 and gap != 0:
                raise Exception('Mismatch in common index {} (when reaching end of l2, sorted)'.format(i))
            if gap < 0:
                raise Exception('Cannot negative pad in index {} (when reaching end of l2, sorted)'.format(i))
            s1 = [l1[i] for i in range(prv[1], tpl[1])]
            s2 = [l2[i] for i in range(prv[2], len(l2))] + \
                 list(repeat(None, gap)) + \
                 [l2[i] for i in range(0, tpl[2])]
            result = result + zip(s1, s2)
        elif tpl[2] - prv[2] != tpl[1] - prv[1]:
            raise Exception('Mismatch in index {} (sorted)'.format(i))
        else:
            s1 = [l1[i] for i in range(prv[1], tpl[1])]
            s2 = [l2[i] for i in range(prv[2], tpl[2])]
            result = result + zip(s1, s2)
    result_is_closed = c1 or c2 or overboard
    if result_is_closed:
        ''' List 1 must perform a whole cycle '''
        prv = tpls[-1]
        tpl = tpls[0]
        gap = (tpl[2] - prv[2]) - (tpl[1] + len(l1) - prv[1])
        if c1:
            if gap != 0:
                raise Exception('Mismatch in when reaching end of l1')
        if gap < 0:
            if tpl[2] < prv[2]:
                logging.debug( 'Gap : {}'.format(gap))
                result.append((l1[-1], l2[-1]))
            else:
                raise Exception('Cannot negative pad when reaching end of l1')
        else:
            s2 = [l2[i] for i in range(prv[2], tpl[2])]
            s1 = [l1[i] for i in range(prv[1], len(l1))] + \
                 list(repeat(None, gap)) + \
                 [l1[i] for i in range(0, tpl[1])]
            result = result + zip(s1, s2)
    else:
        result = padding_zip(l1[0:tpls[0][1]], l2[0:tpls[0][2]], padding_direction=-1) + \
                 result + \
                 padding_zip(l1[tpls[-1][1]:], l2[tpls[-1][2]:])
    return (result_is_closed, list(pairwise_induction(result)))                

In [307]:
def drop(n, it):
    for i in range(n):
        it.next()
    while True:
        yield it.next()

In [308]:
def tokenize_cycle(cycle, anchors):
    (cyclic, members) = cycle
    result = []
    k = 0
    for j in range(len(members)):
        if members[j] in anchors:
            result.append(members[k:j])
            result.append(members[j])
            k = j+1
    if cyclic:
        result[0] = members[k:] + result[0]
    else:
        result[0] = members[k:] + [-1] + result[0]
    return result
def padding_zip(l1, l2, tail=True):
    if len(l1) < len(l2):
        if tail:
            return zip(l1 + [None] * (len(l2) - len(l1)), l2)
        else: 
            return zip([None] * (len(l2) - len(l1)) + l1, l2)
    else:
        if tail:
            return zip(l1, l2 + [None] * (len(l1) - len(l2)))
        else:
            return zip(l1, [None] * (len(l1) - len(l2)) + l2)
def flexi_zip(l1, l2):
    if -1 in l1 and -1 in l2:
        idx1 = l1.index(-1)
        idx2 = l2.index(-1)        
        return padding_zip(l1[0:idx1], l2[0:idx2]) + padding_zip(l1[(idx1+1):], l2[(idx2+1):], False)
    if -1 in l1 and -1 not in l2:
        if (len(l2) - len(l1) + 1) < 0:
            return None
        idx = l1.index(-1)
        return zip(l1[0:idx] + [None] * (len(l2) - len(l1) + 1) + l1[idx+1:], l2)
    elif -1 in l2 and -1 not in l1:
        if (len(l1) - len(l2) + 1) < 0:
            return None
        idx = l2.index(-1)
        return zip(l1, l2[0:idx] + [None] * (len(l1) - len(l2) + 1) + l2[idx+1:])
    elif len(l1) == len(l2):
        return zip(l1, l2)
    else:
        return None
def common_cycle4(s1, s2):
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    common_members = set(s1[1]).intersection(s2[1]) - {None}
    if len(common_members) == 0:
        return None
    t1 = tokenize_cycle(s1, common_members)
    t2 = tokenize_cycle(s2, common_members)
    if len(t1) != len(t2):
        return None
    a_common_member = iter(common_members).next()
    idx1 = t1.index(a_common_member)
    idx2 = t2.index(a_common_member)
    result = []
    cyclic = True
    for j in range(len(t1)):
        token1 = t1[(j + idx1) % len(t1)]
        token2 = t2[(j + idx2) % len(t1)]
        if isinstance(token1, int):
            if not isinstance(token2, int):
                return None
            if token1 != token2:
                return None
            result.append((token1, token2))
            continue
        if isinstance(token2, int):
            return None
        if -1 in token1 and -1 in token2:
            cyclic = False
        fz = flexi_zip(token1, token2)
        if fz is None:
            return None
        result = result + fz
    return (cyclic, list(pairwise_induction(result)))
    

In [309]:
def compare_cyclic(l1, l2):
    from itertools import cycle
    if set(l1) != set(l2) or len(l1) != len(l2):
        return False
    for a,b in zip(l1, drop(l2.index(l1[0]), cycle(l2))):
        if a != b:
            return False
    return True

In [310]:
def index_or_identity(l, i):
    return l[i] if i in l else i

def without(l, indices):
    return [l[i] for i in range(len(l)) if i not in indices]

def without_keys(d, keys):
    return {k: v for k, v in d.iteritems() if k not in keys}

In [311]:
def common_member(l1, l2):    
    for i1 in range(len(l1)):
        x1 = l1[i1]
        if x1 in l2:
            return (i1, l2.index(l1[i1]))
    return (-1, -1)

In [315]:
def test(a, b, expected):
    cc = common_cycle4(a, b)
    (c, result) = cc
    rev = common_cycle4(b, a)
    (cm, mirrored_result) = rev
    (cme, mirrored_expected) = mirror(expected)
    if not compare_cyclic(result, expected[1]) or c != expected[0]:
        logging.debug( '{} and \n{} resulted in \n{} and did not result in \n{}'.format(a, b, cc, expected))
    if not compare_cyclic(mirrored_result, mirrored_expected) or cm != cme:
        logging.debug( '{} and \n{} resulted in \n{} and did not result in \n{}'.format(b, a, mirrored_result, mirror(expected)))        

In [316]:
test((False, [0,1]), (False, [1,2]),(False, [(0,0), (1,1), (2, 2)]))
test((False, [0,1,2,3]), (False, [2,3]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [1,2]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [1,None,3]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [2,None,0]),(True, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [1,2]), (False, [2,3]),(False, [(1,1), (2,2), (3,3)]))
test((False, [0,1]), (True, [0,1]),(True, [(0,0), (1,1)]))
test((True, [11, 2, 0]), (False, [2, 0, 5]), (True, [(11, 5), (2, 2), (0, 0)]))

In [317]:
def replace(graph, merging_table):
    replacer = lambda x: merging_table[x] if x in merging_table else x
    return {
        k: (f, (c, map(replacer, n))) for k, (f, (c, n)) in graph.iteritems()
    }

def scan_for_errors(graph):
    for k, (f, (c, n)) in graph.iteritems():
        if len(n) != len(set(n)):
            print_graph(graph))
            raise Exception('Non unique members in node {}'.format(k))

In [337]:
from functools import partial
from copy import copy
def merge_vertices(graph, i1, i2, merging_table, countdown=100):
    if countdown < 0:
        logging.debug( 'Stack overflow')
        import sys
        sys.exit(1)
    logging.debug( 'Merging {} ({}) into {} ({})'.format(i1, graph[i1][0], i2, graph[i2][0]))
    logging.debug( 'with')
    logging.debug( 'Graph: ')
    print_graph(graph)
    logging.debug( 'Merging Table: ')
    logging.debug( merging_table)
    '''
    vertex of index i1 is merged into i2
    i1 - source index
    i2 - destination index
    '''
    if i1 == i2:
        logging.debug( 'Not merging')
        logging.debug( 'After merging {} into {}'.format(i1, i2))
        logging.debug( graph.keys())
        return graph, merging_table
    f1, (c1, n1) = graph[i1]
    f2, (c2, n2) = graph[i2]
    def mapper(l):
        return [merging_table.get(i, i) for i in l]    
    cc = common_cycle4((c1, mapper(n1)), (c2, mapper(n2)))
    logging.debug( cc)
    if cc is None:
        logging.debug( 'Not merging')
        logging.debug( (i1, c1, mapper(n1)), (i2, c2, mapper(n2)))
        logging.debug( 'After merging {} into {}'.format(i1, i2))
        logging.debug( graph.keys())
        return graph, merging_table
    new_graph = clone_graph(graph)
    new_graph[i2] = (f2, (cc[0], [x[1] for x in cc[1]]))
    local_merging_table = {}
    for src, dst in cc[1]:
        if src != dst:
            local_merging_table[src] = dst
    new_merging_table = copy(local_merging_table)
    new_merging_table.update(merging_table)
    new_merging_table[i1] = i2
    for src, dst in local_merging_table.iteritems():
        if dst in new_graph and src in new_graph:
            new_graph, new_merging_table = merge_vertices(new_graph, src, dst, new_merging_table, countdown=countdown-1)
        else:
            logging.debug( "Skipping {} -> {}".format(src, dst))
#     logging.debug( 'combined_merging_table: {}'.format(combined_merging_table))
#     logging.debug( 'After merging {} into {}'.format(i1, i2))
#     logging.debug( without_keys(replace(new_graph, combined_merging_table), {i1}).keys())
    return (without_keys(replace(new_graph, new_merging_table), {i1}), new_merging_table)

In [338]:
from collections import defaultdict

def progress_graph(initial_graph, path, rules):
    graph = initial_graph
    for c in path:
        logging.debug('Applying {}'.format(c))
        grouped_by_formula = defaultdict(list)
        new_graph = duplicate_and_transform_graph(graph, c)
        logging.debug('Duplicated graph: ')
        print_graph(new_graph)
        for k in sorted(new_graph.keys()):
            (f, x)  = new_graph[k]
            cnt = True
            new_formula = f
            while cnt:
                brk = True
                for fr, to in rules:
                    if fr in new_formula:
                        new_formula = new_formula.replace(fr, to)
                        brk = False
                        logging.debug( 'Reducing {} : {} into {} ({} -> {})'.format(k, f, new_formula, fr, to))
                cnt = cnt and not brk
            grouped_by_formula[new_formula].append(k)
            new_graph[k] = (new_formula, x)    
        logging.debug( 'Reduced graph: ')
        print_graph(new_graph)
        merging_table = {}
        for f, group in grouped_by_formula.iteritems():
            if len(group) == 2:
                a = group[1]
                b = group[0]
                while b in merging_table:
                    b = merging_table[b]
                merging_table[a] = b
            elif len(group) > 2:
                logging.debug( f, group)
                logging.debug( 'A group larger than 2')
                break
        logging.debug( 'Merging the following: {}'.format(merging_table) )
        new_merging_table = merging_table
        for src, dst in merging_table.iteritems():
            if dst in new_graph:
                new_graph, new_merging_table = merge_vertices(new_graph, src, dst, new_merging_table)
                #logging.debug( new_graph.keys())
            else:
                logging.debug( "Not merging {} -> {}".format(src, dst))
        graph = new_graph
        logging.debug( 'Result graph: ')
        print_graph(new_graph)
    return graph

In [341]:
actions = ['a', 'b']
cube_rules = [('aaaa', ''),
         ('bbbb', ''),
         ('abP', 'P')]


cube_lean_initial_graph = {
    0: ('P', (False, [1])),
    1: ('bP', (False, [0, 2])),
    2: ('bbP', (False, [1])),
}

cube_path = 'bbb'
logging.basicConfig(level=logging.DEBUG)
progress_graph(cube_lean_initial_graph, cube_path, cube_rules)

{0: ('P', (False, [1])),
 1: ('bP', (False, [2, 0])),
 2: ('bbP', (True, [1, 5])),
 5: ('bbbP', (False, [2, 0])),
 11: ('P', (False, [5])),
 12: ('bP', (False, [13])),
 13: ('bbP', (False, [14, 12])),
 14: ('bbbP', (True, [13, 17])),
 17: ('P', (False, [14, 12])),
 23: ('bP', (False, [17]))}

In [270]:
actions = ['a', 'b']
bucky_rules = [('aaa', ''),
         ('bbbbb', ''),
         ('ababbP', 'bP')]


bucky_initial_graph = {
    0: ('P', (True, [4, 1, 8])),
    1: ('bP', (True, [0, 2, 5])),
    2: ('bbP', (True, [1, 3, None])),
    3: ('bbbP', (True, [2, 4, None])),
    4: ('bbbbP', (True, [3, 0, None])),
    5: ('aaP', (True, [1, None, 6])),
    6: ('aabP', (True, [5, None, 7])),
    7: ('aP', (True, [6, None, 8])),
    8: ('abP', (True, [7, None, 0])),    
}

bucky_path = 'bbbbabbbaaaab'
bucky = progress_graph(bucky_initial_graph, bucky_path, bucky_rules)

Applying b
Applying b
Applying b
Applying b
Applying a
Applying b
Applying b
Applying b
Applying a
Applying a
Applying a
Applying a
Applying b


In [255]:
actions = ['a', 'b']
bucky_rules = [('aaa', ''),
         ('bbbbb', ''),
         ('ababbP', 'bP')]


bucky_lean_initial_graph = {
    0: ('P', (True, [4, 1, 8])),
    1: ('bP', (True, [0, 2, 5])),
    2: ('bbP', (True, [1, 3, None])),
    3: ('bbbP', (True, [2, 4, None])),
    4: ('bbbbP', (True, [3, 0, None])),
    5: ('aaP', (True, [1, None, 6])),
    6: ('aabP', (True, [5, None, 7])),
    7: ('aP', (True, [6, None, 8])),
    8: ('abP', (True, [7, None, 0])),    
}

bucky_path = 'bbbbabbbaaaab'
progress_graph(bucky_lean_initial_graph, bucky_path, bucky_rules)

Applying b
Applying b
Applying b
Applying b
Applying a
Applying b
Applying b
Applying b
Applying a
Applying a
Applying a
Applying a
Applying b


{0: ('P', (True, [1, 8, 4])),
 1: ('bP', (True, [0, 2, 5])),
 2: ('bbP', (True, [1, 3, 14])),
 3: ('bbbP', (True, [2, 4, 31])),
 4: ('bbbbP', (True, [0, 65, 3])),
 5: ('aaP', (True, [16, 6, 1])),
 6: ('aabP', (True, [270, 7, 5])),
 7: ('aP', (True, [8, 6, 140])),
 8: ('abP', (True, [0, 7, 134])),
 14: ('baaP', (True, [33, 15, 2])),
 15: ('baabP', (True, [16, 14, 542])),
 16: ('baP', (True, [412, 5, 15])),
 31: ('bbaaP', (True, [32, 3, 67])),
 32: ('bbaabP', (True, [33, 31, 1086])),
 33: ('bbaP', (True, [32, 956, 14])),
 65: ('bbbaaP', (True, [66, 4, 135])),
 66: ('bbbaabP', (True, [65, 2174, 67])),
 67: ('bbbaP', (True, [66, 2044, 31])),
 134: ('bbbbaabP', (True, [8, 139, 135])),
 135: ('bbbbaP', (True, [169, 65, 134])),
 139: ('abbbP', (True, [134, 140, 167])),
 140: ('abbbbP', (True, [201, 139, 7])),
 167: ('abbaaP', (True, [203, 168, 139])),
 168: ('abbaabP', (True, [167, 3262, 169])),
 169: ('abbaP', (True, [168, 2174, 135])),
 201: ('abbbaaP', (True, [202, 140, 271])),
 202: ('abb

In [256]:
progress_graph(bucky_initial_graph, bucky_path, bucky_rules)

Applying b
Applying b
Applying b
Applying b
Applying a
Applying b
Applying b
Applying b
Applying a
Applying a
Applying a
Applying a
Applying b


{0: ('P', (True, [1, 8, 4])),
 1: ('bP', (True, [0, 2, 5])),
 2: ('bbP', (True, [1, 3, 14])),
 3: ('bbbP', (True, [2, 4, 31])),
 4: ('bbbbP', (True, [0, 65, 3])),
 5: ('aaP', (True, [16, 6, 1])),
 6: ('aabP', (True, [270, 7, 5])),
 7: ('aP', (True, [8, 6, 140])),
 8: ('abP', (True, [0, 7, 134])),
 14: ('baaP', (True, [33, 15, 2])),
 15: ('baabP', (True, [16, 14, 542])),
 16: ('baP', (True, [412, 5, 15])),
 31: ('bbaaP', (True, [32, 3, 67])),
 32: ('bbaabP', (True, [33, 31, 1086])),
 33: ('bbaP', (True, [32, 956, 14])),
 65: ('bbbaaP', (True, [66, 4, 135])),
 66: ('bbbaabP', (True, [65, 2174, 67])),
 67: ('bbbaP', (True, [66, 2044, 31])),
 134: ('bbbbaabP', (True, [8, 139, 135])),
 135: ('bbbbaP', (True, [169, 65, 134])),
 139: ('abbbP', (True, [134, 140, 167])),
 140: ('abbbbP', (True, [201, 139, 7])),
 167: ('abbaaP', (True, [203, 168, 139])),
 168: ('abbaabP', (True, [167, 3262, 169])),
 169: ('abbaP', (True, [168, 2174, 135])),
 201: ('abbbaaP', (True, [202, 140, 271])),
 202: ('abb

In [45]:
bucky_initial_graph

{0: ('P', (True, [4, 1, 8])),
 1: ('bP', (True, [0, 2, 5])),
 2: ('bbP', (True, [1, 3, None])),
 3: ('bbbP', (True, [2, 4, None])),
 4: ('bbbbP', (True, [3, 0, None])),
 5: ('aaP', (True, [1, None, 6])),
 6: ('aabP', (True, [5, None, 7])),
 7: ('aP', (True, [6, None, 8])),
 8: ('abP', (True, [7, None, 0]))}

In [46]:
def find_cycle(g, r, c, d=0):
    if d < 0:
        return 0
    if r in g[c][1]:
        return d
    else:
        return 1 + find_cycle



In [48]:
actions = ['a', 'b']
dodca_rules = [('aaa', ''),
         ('bbbbb', ''),
         ('aP', 'P'),
         ('bbbbP', 'abP')]


dodca_initial_graph = {
    0: ('P', (True, [4, 1, None])),
    1: ('bP', (True, [0, 2, None])),
    2: ('bbP', (True, [1, 3, None])),
    3: ('bbbP', (True, [2, 4, None])),
    4: ('bbbbP', (True, [3, 0, None])),
}

path = 'aabbbbaabbbabababa'
dodca = progress_graph(dodca_initial_graph, path, dodca_rules)

Applying a
Applying a
Applying b
Applying b
Applying b
Applying b
Applying a
Applying a
Applying b
Applying b
Applying b
Applying a
Applying b
Applying a
Applying b
Applying a
Applying b
Applying a


In [49]:
dodca

{0: ('P', (True, [1, 9, 4])),
 1: ('bP', (True, [0, 2, 18])),
 2: ('bbP', (True, [1, 3, 37])),
 3: ('bbbP', (True, [2, 4, 75])),
 4: ('abP', (True, [0, 7, 3])),
 7: ('abbP', (True, [8, 150, 4])),
 8: ('abbbP', (True, [9, 226, 7])),
 9: ('aabP', (True, [8, 0, 17])),
 17: ('aabbP', (True, [9, 18, 301])),
 18: ('aabbbP', (True, [1, 36, 17])),
 36: ('baabbP', (True, [18, 37, 527])),
 37: ('baabbbP', (True, [2, 74, 36])),
 74: ('bbaabbP', (True, [75, 1055, 37])),
 75: ('bbaabbbP', (True, [74, 3, 150])),
 150: ('bbbaabbP', (True, [225, 75, 7])),
 225: ('abbaabbP', (True, [226, 1055, 150])),
 226: ('abbaabbbP', (True, [8, 301, 225])),
 301: ('abbbaabbP', (True, [17, 527, 226])),
 527: ('aabbaabbP', (True, [301, 36, 1055])),
 1055: ('baabbaabbP', (True, [225, 527, 74]))}